In [14]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [15]:
x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
x.grad  # The default value is None

In [16]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [17]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [18]:
x.grad == 4 * x

tensor([True, True, True, True])

In [26]:
# PyTorch accumulates the gradient in default, we need to clear the previous
# values
x.grad.zero_()
y = x.sum()
print(x.sum())
y.backward()
x.grad

tensor(6., grad_fn=<SumBackward0>)


tensor([1., 1., 1., 1.])

## Backward for Non-Scalar Variables

In [27]:
x.grad.zero_()
print(x)
y = x * x
# y.backward(torch.ones(len(x))) equivalent to the below
print(y)
print(y.sum())
y.sum().backward()
x.grad

tensor([0., 1., 2., 3.], requires_grad=True)
tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)
tensor(14., grad_fn=<SumBackward0>)


tensor([0., 2., 4., 6.])

In [29]:
# Invoking `backward` on a non-scalar requires passing in a `gradient` argument
# which specifies the gradient of the differentiated function w.r.t `self`.
# In our case, we simply want to sum the partial derivatives, so passing
# in a gradient of ones is appropriate
x.grad.zero_()
y = x * x
y.backward(torch.ones(len(x))) #equivalent to the below
print(len(x))
print(torch.ones(len(x)))
#y.sum().backward()
x.grad

4
tensor([1., 1., 1., 1.])


tensor([0., 2., 4., 6.])

## Detaching Computation

In [30]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [31]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Computing the Gradient of Python Control Flow

In the control flow example where we calculate the derivative of d with respect to a, what would happen if we changed the variable a to a random vector or matrix. At this point, the result of the calculation f(a) is no longer a scalar. What happens to the result? How do we analyze this?

In [32]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [33]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [34]:
a.grad == d / a

tensor(True)

In [38]:
demo = torch.tensor([3,4], dtype = torch.float64)
demo.norm()

tensor(5., dtype=torch.float64)

In [39]:
demo = torch.tensor([3,4], dtype = torch.float32)
demo.norm()

tensor(5.)